In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read the datasets into pandas DataFrames
aviation_data = pd.read_csv('data/AviationData.csv', encoding='latin-1')
state_codes = pd.read_csv('data/USState_Codes.csv')
aviation_data.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_10508\3358021668.py:2: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  aviation_data = pd.read_csv('data/AviationData.csv', encoding='latin-1')


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [3]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [4]:
aviation_data.columns = aviation_data.columns.str.lower().str.replace('.', '_', regex=False)
aviation_data.columns

Index(['event_id', 'investigation_type', 'accident_number', 'event_date',
       'location', 'country', 'latitude', 'longitude', 'airport_code',
       'airport_name', 'injury_severity', 'aircraft_damage',
       'aircraft_category', 'registration_number', 'make', 'model',
       'amateur_built', 'number_of_engines', 'engine_type', 'far_description',
       'schedule', 'purpose_of_flight', 'air_carrier', 'total_fatal_injuries',
       'total_serious_injuries', 'total_minor_injuries', 'total_uninjured',
       'weather_condition', 'broad_phase_of_flight', 'report_status',
       'publication_date'],
      dtype='object')

In [5]:
aviation_data.isna().sum()

event_id                      0
investigation_type            0
accident_number               0
event_date                    0
location                     52
country                     226
latitude                  54507
longitude                 54516
airport_code              38757
airport_name              36185
injury_severity            1000
aircraft_damage            3194
aircraft_category         56602
registration_number        1382
make                         63
model                        92
amateur_built               102
number_of_engines          6084
engine_type                7096
far_description           56866
schedule                  76307
purpose_of_flight          6192
air_carrier               72241
total_fatal_injuries      11401
total_serious_injuries    12510
total_minor_injuries      11933
total_uninjured            5912
weather_condition          4492
broad_phase_of_flight     27165
report_status              6384
publication_date          13771
dtype: i

In [6]:
aviation_data['aircraft_category'].value_counts()

aircraft_category
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [7]:
aviation_data['aircraft_category'] = aviation_data['aircraft_category'].replace({'UNK': 'Unknown', 'ULTR': 'Ultralight'})

aviation_data['aircraft_category'].fillna('Unknown', inplace=True)

In [8]:
aviation_data['aircraft_category'].value_counts()

aircraft_category
Unknown              56618
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              31
WSFT                     9
Powered-Lift             5
Blimp                    4
Rocket                   1
Name: count, dtype: int64

In [9]:
aviation_data.drop(columns=[
    'event_id', 'accident_number', 'latitude', 'longitude', 
    'airport_code', 'airport_name', 'registration_number', 
    'far_description', 'schedule', 'air_carrier', 
    'report_status', 'publication_date'
], inplace=True)

aviation_data.isna().sum()

investigation_type            0
event_date                    0
location                     52
country                     226
injury_severity            1000
aircraft_damage            3194
aircraft_category             0
make                         63
model                        92
amateur_built               102
number_of_engines          6084
engine_type                7096
purpose_of_flight          6192
total_fatal_injuries      11401
total_serious_injuries    12510
total_minor_injuries      11933
total_uninjured            5912
weather_condition          4492
broad_phase_of_flight     27165
dtype: int64

In [10]:
aviation_data.describe()

,number_of_engines,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


In [11]:
# Filling missing data in the 'location' column with 'Unknown'
aviation_data['location'].fillna('Unknown', inplace=True)

In [12]:
# Impute missing country values with unknown
aviation_data['country'].fillna('Unknown', inplace=True)

In [13]:
aviation_data['make'].value_counts().head(45)

make
Cessna                            22227
Piper                             12029
CESSNA                             4922
Beech                              4330
PIPER                              2841
Bell                               2134
Boeing                             1594
BOEING                             1151
Grumman                            1094
Mooney                             1092
BEECH                              1042
Robinson                            946
Bellanca                            886
Hughes                              795
Schweizer                           629
Air Tractor                         595
BELL                                588
Mcdonnell Douglas                   526
Aeronca                             487
Maule                               445
Champion                            428
De Havilland                        382
Aero Commander                      360
Stinson                             348
Aerospatiale                       

In [14]:
# drop 3(injury_surverity), 4(aircraft_damage), 5(make and model)
aviation_data['engine_type'].value_counts()

engine_type
Reciprocating      69530
Turbo Shaft         3609
Turbo Prop          3391
Turbo Fan           2481
Unknown             2051
Turbo Jet            703
Geared Turbofan       12
Electric              10
LR                     2
NONE                   2
Hybrid Rocket          1
UNK                    1
Name: count, dtype: int64

In [15]:
# Impute 'location' and 'country' with 'Unknown'
aviation_data['location'].fillna('Unknown', inplace=True)
aviation_data['country'].fillna('Unknown', inplace=True)

# Drop rows with null values in critical columns
aviation_data.dropna(subset=['injury_severity', 'aircraft_damage', 'make', 'model'], inplace=True)

# Fill 'amateur_built', 'number_of_engines', and 'engine_type' with their mode
for column in ['amateur_built', 'number_of_engines', 'engine_type']:
    mode_value = aviation_data[column].mode()[0]
    aviation_data[column].fillna(mode_value, inplace=True)

# Check remaining nulls in key columns
aviation_data[['location', 'country', 'injury_severity', 'aircraft_damage', 
               'make', 'model', 'amateur_built', 'number_of_engines', 
               'engine_type']].isna().sum()


location             0
country              0
injury_severity      0
aircraft_damage      0
make                 0
model                0
amateur_built        0
number_of_engines    0
engine_type          0
dtype: int64

In [16]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85121 entries, 0 to 88886
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   investigation_type      85121 non-null  object 
 1   event_date              85121 non-null  object 
 2   location                85121 non-null  object 
 3   country                 85121 non-null  object 
 4   injury_severity         85121 non-null  object 
 5   aircraft_damage         85121 non-null  object 
 6   aircraft_category       85121 non-null  object 
 7   make                    85121 non-null  object 
 8   model                   85121 non-null  object 
 9   amateur_built           85121 non-null  object 
 10  number_of_engines       85121 non-null  float64
 11  engine_type             85121 non-null  object 
 12  purpose_of_flight       81038 non-null  object 
 13  total_fatal_injuries    74223 non-null  float64
 14  total_serious_injuries  72953 non-null  flo

In [17]:
aviation_data['broad_phase_of_flight'].value_counts().head(25)

broad_phase_of_flight
Landing        15185
Takeoff        12332
Cruise          9936
Maneuvering     8095
Approach        6425
Climb           1913
Taxi            1861
Descent         1724
Go-around       1346
Standing         752
Unknown          544
Other            113
Name: count, dtype: int64

In [18]:
aviation_data['purpose_of_flight'].fillna('Unknown', inplace=True)

In [19]:
aviation_data['total_fatal_injuries'].fillna(0, inplace=True)

In [20]:
aviation_data['total_serious_injuries'].fillna(0, inplace=True)

In [21]:
aviation_data['total_minor_injuries'].fillna(0, inplace=True)

In [22]:
median_uninjured = aviation_data['total_uninjured'].median()
aviation_data['total_uninjured'].fillna(median_uninjured, inplace=True)

In [23]:
# Standardize "UNK" and "Unk" to "Unknown"
aviation_data['weather_condition'].replace(['UNK', 'Unk'], 'Unknown', inplace=True)

# Impute missing values with "Unknown"
aviation_data['weather_condition'].fillna('Unknown', inplace=True)


In [24]:
aviation_data['broad_phase_of_flight'].fillna('Unknown', inplace=True)

In [25]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85121 entries, 0 to 88886
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   investigation_type      85121 non-null  object 
 1   event_date              85121 non-null  object 
 2   location                85121 non-null  object 
 3   country                 85121 non-null  object 
 4   injury_severity         85121 non-null  object 
 5   aircraft_damage         85121 non-null  object 
 6   aircraft_category       85121 non-null  object 
 7   make                    85121 non-null  object 
 8   model                   85121 non-null  object 
 9   amateur_built           85121 non-null  object 
 10  number_of_engines       85121 non-null  float64
 11  engine_type             85121 non-null  object 
 12  purpose_of_flight       85121 non-null  object 
 13  total_fatal_injuries    85121 non-null  float64
 14  total_serious_injuries  85121 non-null  flo

In [29]:
# List of non-numeric columns
non_numeric_columns = [
    'investigation_type', 'event_date', 'location', 'country', 
    'injury_severity', 'aircraft_damage', 'aircraft_category', 
    'make', 'model', 'amateur_built', 'engine_type', 
    'purpose_of_flight', 'weather_condition', 'broad_phase_of_flight'
]

# Display unique values for each non-numeric column
for column in non_numeric_columns:
    print(f"Unique values in {column} - {aviation_data[column].nunique()} values:")
    print(aviation_data[column].unique())
    print("\n")


Unique values in investigation_type - 2 values:
['Accident' 'Incident']


Unique values in event_date - 14706 values:
['1948-10-24' '1962-07-19' '1974-08-30' ... '2022-12-15' '2022-12-16'
 '2022-12-26']


Unique values in location - 26630 values:
['MOOSE CREEK, ID' 'BRIDGEPORT, CA' 'Saltville, VA' ...
 'Guantanamo Bay, OF' 'Regent, ND' 'Woodsfield, OH']


Unique values in country - 206 values:
['United States' 'Unknown' 'GULF OF MEXICO' 'Puerto Rico' 'ATLANTIC OCEAN'
 'HIGH ISLAND' 'Bahamas' 'MISSING' 'Pakistan' 'Angola' 'Germany'
 'Korea, Republic Of' 'Martinique' 'American Samoa' 'PACIFIC OCEAN'
 'Canada' 'Bolivia' 'Mexico' 'Dominica' 'Netherlands Antilles' 'Iceland'
 'Greece' 'Guam' 'Australia' 'CARIBBEAN SEA' 'West Indies' 'Japan'
 'Philippines' 'Venezuela' 'Bermuda' 'San Juan Islands' 'Colombia'
 'El Salvador' 'United Kingdom' 'British Virgin Islands' 'Netherlands'
 'Mozambique' 'Jamaica' 'Panama' 'Costa Rica' 'Norway' 'Hong Kong'
 'Portugal' 'Malaysia' 'Turks And Caicos Islands'


In [37]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85121 entries, 0 to 88886
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   investigation_type      85121 non-null  object        
 1   event_date              85121 non-null  datetime64[ns]
 2   location                85121 non-null  object        
 3   country                 85121 non-null  object        
 4   injury_severity         85121 non-null  object        
 5   aircraft_damage         85121 non-null  object        
 6   aircraft_category       85121 non-null  object        
 7   make                    85121 non-null  object        
 8   model                   85121 non-null  object        
 9   amateur_built           85121 non-null  object        
 10  number_of_engines       85121 non-null  float64       
 11  engine_type             85121 non-null  object        
 12  purpose_of_flight       85121 non-null  object     

In [34]:
# Step 1: Convert event_date to datetime
aviation_data['event_date'] = pd.to_datetime(aviation_data['event_date'])

# Step 2: Extract year and month
aviation_data['year'] = aviation_data['event_date'].dt.year
aviation_data['month'] = aviation_data['event_date'].dt.month

# Display the updated DataFrame to check the new columns
print(aviation_data[['event_date', 'year', 'month']].tail(20))

      event_date  year  month
88842 2022-12-04  2022     12
88843 2022-12-04  2022     12
88844 2022-12-04  2022     12
88845 2022-12-05  2022     12
88846 2022-12-05  2022     12
88847 2022-12-05  2022     12
88848 2022-12-05  2022     12
88854 2022-12-07  2022     12
88856 2022-12-08  2022     12
88857 2022-12-08  2022     12
88858 2022-12-09  2022     12
88859 2022-12-09  2022     12
88861 2022-12-10  2022     12
88865 2022-12-12  2022     12
88867 2022-12-12  2022     12
88869 2022-12-13  2022     12
88873 2022-12-14  2022     12
88876 2022-12-15  2022     12
88877 2022-12-16  2022     12
88886 2022-12-26  2022     12


In [35]:
# Extract month names using strftime
# aviation_data['month_name'] = aviation_data['event_date'].dt.strftime('%b')  # Abbreviated month name (e.g., Jan, Feb)
aviation_data['month_name'] = aviation_data['event_date'].dt.strftime('%B')  # Full month name (e.g., January, February)

# Display the updated DataFrame
print(aviation_data[['month', 'month_name']].head(20))


    month month_name
0      10    October
1       7       July
2       8     August
3       6       June
4       8     August
5       9  September
6       8     August
7       1    January
8       1    January
9       1    January
10      1    January
11      1    January
12      1    January
13      1    January
14      1    January
15      1    January
16      1    January
17      1    January
18      1    January
19      1    January


In [38]:
# Update the mapping dictionary to use "Ivory Coast"
country_mapping = {
    'United States': 'USA',
    'Unknown': 'Not Specified',
    'MISSING': 'Not Specified',
    'GULF OF MEXICO': 'Other',
    'PACIFIC OCEAN': 'Other',
    'ATLANTIC OCEAN': 'Other',
    'Saint Vincent and the Grenadines': 'St Vincent And The Grenadines',
    'Ivory Coast': 'Ivory Coast',
    'Côte d\'Ivoire': 'Ivory Coast',
    "Cote D'ivoire": 'Ivory Coast',
    'South Korean': 'South Korea',
    'Republic of North Macedonia': 'North Macedonia',
    'Antigua And Barbuda': 'Antigua and Barbuda'
}

# Apply the updated mapping
aviation_data['country'] = aviation_data['country'].replace(country_mapping)

# Check the unique values after standardization
print(aviation_data['country'].value_counts().head(10))


country
USA               80177
Brazil              311
Canada              290
Mexico              281
Not Specified       234
United Kingdom      209
Australia           199
Bahamas             199
France              178
Germany             166
Name: count, dtype: int64


In [43]:
aviation_data['injury_severity'].value_counts()

injury_severity
Non-Fatal     65720
Fatal(1)       6066
Fatal          5132
Fatal(2)       3699
Incident       1389
              ...  
Fatal(43)         1
Fatal(60)         1
Fatal(28)         1
Fatal(143)        1
Fatal(153)        1
Name: count, Length: 107, dtype: int64

In [45]:
# Dropping the injury severity column because we already have columns like total_fatal_injuries
aviation_data.drop('injury_severity', axis=1, inplace=True)

In [46]:
aviation_data['aircraft_damage'].value_counts()

aircraft_damage
Substantial    63916
Destroyed      18535
Minor           2578
Unknown           92
Name: count, dtype: int64

In [47]:
# Drop rows with "Unknown" in the 'aircraft_damage' column
aviation_data = aviation_data[aviation_data['aircraft_damage'] != 'Unknown']

# Verify the changes
print(aviation_data['aircraft_damage'].value_counts())

aircraft_damage
Substantial    63916
Destroyed      18535
Minor           2578
Name: count, dtype: int64


In [49]:
aviation_data['aircraft_category'].value_counts()

aircraft_category
Unknown              54798
Airplane             25828
Helicopter            3296
Glider                 503
Gyrocraft              173
Weight-Shift           160
Balloon                135
Powered Parachute       88
Ultralight              30
WSFT                     9
Blimp                    4
Powered-Lift             4
Rocket                   1
Name: count, dtype: int64

In [50]:
aviation_data['investigation_type'].value_counts()

investigation_type
Accident    83129
Incident     1900
Name: count, dtype: int64

In [51]:
# Create a DataFrame with rows where 'aircraft_category' is not 'Unknown'
known_aircraft_data = aviation_data[aviation_data['aircraft_category'] != 'Unknown']

# Create a DataFrame with rows where 'aircraft_category' is 'Unknown'
unknown_aircraft_data = aviation_data[aviation_data['aircraft_category'] == 'Unknown']

In [60]:
aviation_data['make'].value_counts().head(40)

make
Cessna                         22078
Piper                          11953
CESSNA                          4716
Beech                           4282
PIPER                           2755
Bell                            2091
Mooney                          1086
Grumman                         1085
Boeing                          1049
BEECH                           1000
Robinson                         941
Bellanca                         883
Hughes                           786
Schweizer                        624
Air Tractor                      593
BELL                             547
Aeronca                          485
Maule                            443
Champion                         426
Mcdonnell Douglas                389
BOEING                           388
Aero Commander                   356
De Havilland                     351
Stinson                          348
Rockwell                         330
Taylorcraft                      320
Luscombe                         

In [73]:
# Standardize 'make' column: convert to title case and strip whitespaces
aviation_data['make'] = aviation_data['make'].str.title().str.strip()

# Check the unique values and the top 40 after standardization
print(aviation_data['make'].value_counts().iloc[0:10])

make
Cessna      26794
Piper       14708
Beech        5282
Bell         2638
Boeing       1437
Mooney       1318
Robinson     1188
Grumman      1162
Bellanca     1040
Hughes        918
Name: count, dtype: int64


In [74]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85029 entries, 0 to 88886
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   investigation_type      85029 non-null  object        
 1   event_date              85029 non-null  datetime64[ns]
 2   location                85029 non-null  object        
 3   country                 85029 non-null  object        
 4   aircraft_damage         85029 non-null  object        
 5   aircraft_category       85029 non-null  object        
 6   make                    85029 non-null  object        
 7   model                   85029 non-null  object        
 8   amateur_built           85029 non-null  object        
 9   number_of_engines       85029 non-null  float64       
 10  engine_type             85029 non-null  object        
 11  purpose_of_flight       85029 non-null  object        
 12  total_fatal_injuries    85029 non-null  float64    

In [79]:
# Filter for USA cases
usa_data = aviation_data[aviation_data['country'] == 'USA'].copy()
# Extract state abbreviation from location (assuming format 'City, State')
usa_data['state_abbr'] = usa_data['location'].str.split(',').str[-1].str.strip()

# Create a dictionary for mapping abbreviations to state names
state_mapping = dict(zip(state_codes['Abbreviation'], state_codes['US_State']))

# Add the full state name to the USA DataFrame
usa_data['state_name'] = usa_data['state_abbr'].map(state_mapping)
print(usa_data[['location', 'state_abbr', 'state_name']].head(10))


           location state_abbr     state_name
0   MOOSE CREEK, ID         ID          Idaho
1    BRIDGEPORT, CA         CA     California
2     Saltville, VA         VA       Virginia
3        EUREKA, CA         CA     California
4        Canton, OH         OH           Ohio
5        BOSTON, MA         MA  Massachusetts
6        COTTON, MN         MN      Minnesota
7       PULLMAN, WA         WA     Washington
8  EAST HANOVER, NJ         NJ     New Jersey
9  JACKSONVILLE, FL         FL        Florida


In [80]:
# Group by state_name and count the number of occurrences
state_case_counts = usa_data['state_name'].value_counts()

# Convert to a DataFrame for better readability
state_case_counts_df = state_case_counts.reset_index()
state_case_counts_df.columns = ['State', 'Case Count']

# Display the top states with the most cases
print(state_case_counts_df.head(10))

        State  Case Count
0  California        8664
1       Texas        5761
2     Florida        5665
3      Alaska        5603
4     Arizona        2778
5    Colorado        2611
6  Washington        2565
7    Michigan        1969
8     Georgia        1955
9    Illinois        1919


In [81]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85029 entries, 0 to 88886
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   investigation_type      85029 non-null  object        
 1   event_date              85029 non-null  datetime64[ns]
 2   location                85029 non-null  object        
 3   country                 85029 non-null  object        
 4   aircraft_damage         85029 non-null  object        
 5   aircraft_category       85029 non-null  object        
 6   make                    85029 non-null  object        
 7   model                   85029 non-null  object        
 8   amateur_built           85029 non-null  object        
 9   number_of_engines       85029 non-null  float64       
 10  engine_type             85029 non-null  object        
 11  purpose_of_flight       85029 non-null  object        
 12  total_fatal_injuries    85029 non-null  float64    

In [82]:
usa_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80127 entries, 0 to 88886
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   investigation_type      80127 non-null  object        
 1   event_date              80127 non-null  datetime64[ns]
 2   location                80127 non-null  object        
 3   country                 80127 non-null  object        
 4   aircraft_damage         80127 non-null  object        
 5   aircraft_category       80127 non-null  object        
 6   make                    80127 non-null  object        
 7   model                   80127 non-null  object        
 8   amateur_built           80127 non-null  object        
 9   number_of_engines       80127 non-null  float64       
 10  engine_type             80127 non-null  object        
 11  purpose_of_flight       80127 non-null  object        
 12  total_fatal_injuries    80127 non-null  float64    

In [84]:
df_usa = usa_data.dropna()
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80055 entries, 0 to 88886
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   investigation_type      80055 non-null  object        
 1   event_date              80055 non-null  datetime64[ns]
 2   location                80055 non-null  object        
 3   country                 80055 non-null  object        
 4   aircraft_damage         80055 non-null  object        
 5   aircraft_category       80055 non-null  object        
 6   make                    80055 non-null  object        
 7   model                   80055 non-null  object        
 8   amateur_built           80055 non-null  object        
 9   number_of_engines       80055 non-null  float64       
 10  engine_type             80055 non-null  object        
 11  purpose_of_flight       80055 non-null  object        
 12  total_fatal_injuries    80055 non-null  float64    

In [86]:
# Drop 'month' and 'state_abbr' columns from df_usa
df_usa.drop(columns=['month', 'state_abbr'], inplace=True)

# Verify the changes
print(df_usa.columns)

Index(['investigation_type', 'event_date', 'location', 'country',
       'aircraft_damage', 'aircraft_category', 'make', 'model',
       'amateur_built', 'number_of_engines', 'engine_type',
       'purpose_of_flight', 'total_fatal_injuries', 'total_serious_injuries',
       'total_minor_injuries', 'total_uninjured', 'weather_condition',
       'broad_phase_of_flight', 'year', 'month_name', 'state_name'],
      dtype='object')


C:\Users\HP\AppData\Local\Temp\ipykernel_10508\3052871411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_usa.drop(columns=['month', 'state_abbr'], inplace=True)
